In [1]:
import os
import json
import numpy as np
import pandas as pd
from argparse import Namespace
from glob import glob

In [2]:
SVY_IN_DIR = 'data/GiveDirectly/Survey/household.dta'
SAT_IN_DIR = 'data/GiveDirectly/Pred/infer/'
OUT_DIR = 'data/GiveDirectly/merged.csv'

# set up visualization cfg
cfg = Namespace()
cfg.visual_score_cutoff = 0.9

In [3]:
df = pd.read_stata(SVY_IN_DIR)
df = df.dropna(subset=['s19_gps_latitude', 's19_gps_longitude'])

In [4]:
# read satellite predictions
# from json annotations
df_sat = []
for file_name in glob(os.path.join(SAT_IN_DIR, '*.json')):
    with open(file_name, 'r') as f:
        df_sat += json.load(f)
df_sat = pd.DataFrame(df_sat)

# drop low score predictions
df_sat = df_sat.loc[df_sat['score'] > cfg.visual_score_cutoff, :]

In [5]:
df_sat['category_id'].replace(
    {1: 'thatched', 2: 'metal', 3: 'colored'}, inplace=True)

In [6]:
df_sat

,segmentation,bbox,score,area,image_id_str,image_id,category_id
0,"{'size': [800, 800], 'counts': 'co]7h0og0:L4M2...","[303.96514892578125, 483.6033935546875, 61.135...",0.999874,2616.0,601040506010-082,3490,metal
1,"{'size': [800, 800], 'counts': '_db96gh04M3M2M...","[390.64776611328125, 350.5533752441406, 69.738...",0.999759,3197.0,601040506010-082,3490,metal
2,"{'size': [800, 800], 'counts': 'kT[17eh06K3M4J...","[55.012939453125, 165.5957794189453, 38.862304...",0.995076,1060.0,601040506010-082,3490,thatched
4,"{'size': [800, 800], 'counts': 'Zed:5ih04L3N3M...","[431.2185974121094, 0.0, 110.86050415039062, 1...",0.999966,6870.0,601010105009-032,216,metal
5,"{'size': [800, 800], 'counts': '\ZY89fh03N3L3N...","[339.1448974609375, 470.09857177734375, 89.540...",0.999945,4315.0,601010105009-032,216,metal
...,...,...,...,...,...,...,...
36945,"{'size': [800, 800], 'counts': 'og_b0^1bg000O1...","[756.798095703125, 348.59075927734375, 42.9449...",0.999768,2062.0,601050103008-097,4078,metal
36946,"{'size': [800, 800], 'counts': 'eSe54kh02N2N3N...","[231.73077392578125, 641.6854248046875, 48.781...",0.999517,1463.0,601050103008-097,4078,metal
36947,"{'size': [800, 800], 'counts': '`SS63kh05L2N2N...","[248.94029235839844, 551.2299194335938, 86.294...",0.999350,4417.0,601050103008-097,4078,metal
36948,"{'size': [800, 800], 'counts': 'Rid::dh04L4M2N...","[435.94049072265625, 120.67565155029297, 95.66...",0.999129,4772.0,601050103008-097,4078,metal


In [7]:
# grouping
df_cat = df_sat.groupby(['image_id_str', 'category_id']).agg(
    sat_house=pd.NamedAgg(column='area', aggfunc='count'),
    sat_size_mean=pd.NamedAgg(column='area', aggfunc=np.nanmean),
    sat_size_sum=pd.NamedAgg(column='area', aggfunc=np.nansum)
).reset_index()

In [8]:
df_all = df_sat.groupby(['image_id_str']).agg(
    sat_house=pd.NamedAgg(column='area', aggfunc='count'),
    sat_size_mean=pd.NamedAgg(column='area', aggfunc=np.nanmean),
    sat_size_sum=pd.NamedAgg(column='area', aggfunc=np.nansum)
).reset_index()

In [9]:
df_sat = pd.concat([df_cat, df_all.assign(category_id='all')], sort=True)

In [10]:
# scale areas / distances
df_sat[[col for col in df_sat.columns if col.startswith('sat_size')]] *= (
    (0.298 ** 2) * (640 ** 2) / (800 ** 2) * np.cos(23 / 180 * np.pi))  # in sq meters

In [11]:
df_sat = df_sat.pivot(
    index='image_id_str', columns='category_id',
    values=['sat_house', 'sat_size_mean', 'sat_size_sum'])

In [12]:
df_sat_columns = ['_'.join(cols[::-1]) for cols in df_sat.columns]
df_sat.columns = df_sat_columns

In [13]:
df = pd.merge(df, df_sat, how='left', left_on='s1_hhid_key', right_on='image_id_str')

In [14]:
cols = [col for col in df_sat_columns if col.endswith(('sat_house', 'sat_size_sum'))]
df[cols] = df[cols].fillna(0)

In [15]:
df.to_csv(OUT_DIR, index=False)

In [16]:
df

,s1_hhid_key,s6_q1_haselectricity,s6_q1a_electype,s6_q6_acresowned,s6_q6b_agacresowned,s6_q7_rentedland,s6_q7a_acresrented,s6_q7b_monthsrented,s6_q7ci_landrent,s6_q8_landrenting,...,metal_sat_house,thatched_sat_house,all_sat_size_mean,colored_sat_size_mean,metal_sat_size_mean,thatched_sat_size_mean,all_sat_size_sum,colored_sat_size_sum,metal_sat_size_sum,thatched_sat_size_sum
0,601010101002-002,2.0,,2.000,1.500,2.0,NaN,NaN,NaN,2.0,...,4.0,2.0,191.732456,371.499384,153.823555,177.666794,1342.127191,371.499384,615.294219,355.333589
1,601010101002-031,2.0,,0.500,0.500,2.0,NaN,NaN,NaN,1.0,...,4.0,3.0,156.934517,NaN,190.628204,112.009602,1098.541622,0.000000,762.512817,336.028805
2,601010101002-041,2.0,,1.000,0.500,2.0,NaN,NaN,NaN,2.0,...,1.0,0.0,502.552187,NaN,502.552187,NaN,502.552187,0.000000,502.552187,0.000000
3,601010101002-054,2.0,,0.500,0.500,2.0,NaN,NaN,NaN,2.0,...,3.0,3.0,46.021071,NaN,50.938821,41.103321,276.126425,0.000000,152.816462,123.309963
4,601010101002-060,1.0,1,0.500,0.500,2.0,NaN,NaN,NaN,2.0,...,12.0,2.0,297.086653,NaN,315.542540,186.351331,4159.213138,0.000000,3786.510475,372.702663
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5228,601050504009-051,2.0,,0.250,0.250,2.0,NaN,NaN,NaN,2.0,...,2.0,1.0,121.077793,NaN,102.880374,157.472630,363.233378,0.000000,205.760749,157.472630
5229,601050504009-070,2.0,,0.250,0.250,2.0,NaN,NaN,NaN,2.0,...,4.0,3.0,125.873471,NaN,158.362010,82.555419,881.114296,0.000000,633.448041,247.666256
5230,601050504009-098,2.0,,0.625,0.625,2.0,NaN,NaN,NaN,1.0,...,3.0,2.0,182.793810,NaN,133.441923,256.821641,913.969051,0.000000,400.325769,513.643282
5231,601050504009-143,2.0,,0.500,0.500,2.0,NaN,NaN,NaN,2.0,...,5.0,1.0,132.181345,146.538484,133.229169,112.585083,925.269412,146.538484,666.145846,112.585083
